# Les optimisation en ordonnancement.

# Introduction.

Dans ce notebook, je vais principalement traiter des problèmes d'ordonnancement en usine et en atelier, cependant, on peut transposer les modèles mathématiques à d'autres cas d'utilisation.

Les principaux modèles sont ( En Anglais ) :

- Machine unique ( Single machine sheduling )
- Machines parallèles 
- Ateliers à cheminement unique (Flow Shop)
- Ateliers à cheminements multiples (Job Shop)



A venir 

Single machine scheduling
Flow Shop
Job Shop
...